<a href="https://colab.research.google.com/github/VMBoehm/SDSS_PAE/blob/main/LSTM_AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Splitting the dataset into training, validation and test.
Further redshift cuts


In [3]:
import tensorflow as tf
tf.__version__
tf.config.list_physical_devices('GPU')


import numpy as np
import scipy
import matplotlib.pyplot as plt
import os
import logging
import sys
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Layer, Reshape, LeakyReLU, BatchNormalization, Dense, Flatten, Input,Dropout
import random
import numpy as np
import pickle
import pandas as pd

In [52]:
### settings
# user defined span (following Yip et al and Portillo et al)
root_models     = '/global/cscratch1/sd/vboehm/Models/SDSS_AE/'

root_model_data = '/global/cscratch1/sd/vboehm/Datasets/sdss/by_model/'
root_encoded    = '/global/cscratch1/sd/vboehm/Datasets/encoded/sdss/'
root_decoded    = '/global/cscratch1/sd/vboehm/Datasets/decoded/sdss/'
root_data       = '/global/cscratch1/sd/vboehm/Datasets'

root_prepped    = os.path.join(root_data,'sdss/prepped')

wlmin, wlmax    = (3388,8318)
fixed_num_bins  = 1000
min_SN          = 50
min_z           = 0.05
max_z           = 0.36
label           = 'galaxies_quasars_bins%d_wl%d-%d'%(fixed_num_bins,wlmin,wlmax)
label_          = label+'_minz%s_maxz%s_minSN%d'%(str(int(min_z*100)).zfill(3),str(int(max_z*100)).zfill(3),min_SN)


seed            = 10

In [53]:
### load data and join

res_fluxes, res_masks, res_inv_vars = np.load(os.path.join(root_prepped,'prepped_data_spectra_%s_QSO_GAL.npy'%(label_)))

redshifts, SNs, ras, decs, category, sublabel, MJD, plate_id, fiber= np.load(os.path.join(root_prepped,'prepped_data_prop_%s_QSO_GAL.npy'%(label_)))


In [54]:
print(len(res_fluxes))

349104


In [55]:
indices = np.arange(len(res_fluxes))

In [56]:
np.random.seed(seed)
np.random.shuffle(indices)

In [57]:
def reshuffle(list_, indices):
    res = []
    for item_ in list_:
        res.append(item_[indices])
    return res

In [58]:
res_fluxes, res_masks, res_inv_vars, redshifts, SNs, ras, decs, category, sublabel, MJD, plate_id, fiber = reshuffle([res_fluxes, res_masks, res_inv_vars, redshifts, SNs, ras, decs, category, sublabel,MJD, plate_id, fiber], indices)

In [59]:
# def redshift_bin(min_z, max_z, z_array, arrays):
#     num  = len(arrays)
#     z_array = np.asarray(z_array, dtype=np.float32)
#     indx = np.where((z_array>min_z)*(z_array<max_z))[0]
#     arrays_new = [arrays[ii][indx] for ii in range(num)]
#     print('new datasize: ', len(indx))
#     return arrays_new

# res_fluxes, res_masks, res_inv_vars, redshifts, SNs, ras, decs, category, sublabel, MJD, plate_id, fiber = redshift_bin(min_z_cut,max_z, redshifts, [res_fluxes, res_masks, res_inv_vars, redshifts, SNs, ras, decs, category, sublabel, MJD, plate_id, fiber])

In [60]:
DATASET_SIZE = len(res_fluxes)
train_size   = int(0.6 * DATASET_SIZE)
test_size    = int(0.2 * DATASET_SIZE)
valid_size   = int(0.2 * DATASET_SIZE)
indices      = np.arange(DATASET_SIZE)

In [61]:
mean=np.mean(res_fluxes)
std = np.std(res_fluxes)

#### if only divide by mean
std  = mean
mean = 0

In [62]:
train_data   = np.expand_dims((res_fluxes[:train_size]-mean)/std,-1)
valid_data   = np.expand_dims((res_fluxes[train_size:train_size+valid_size]-mean)/std,-1)
test_data    = np.expand_dims((res_fluxes[train_size+valid_size::]-mean)/std,-1)

train_mask   = np.expand_dims(res_masks[:train_size],-1)
valid_mask   = np.expand_dims(res_masks[train_size:train_size+valid_size],-1)
test_mask    = np.expand_dims(res_masks[train_size+valid_size::],-1)

train_noise  = np.expand_dims(res_inv_vars[:train_size],-1)*std**2
valid_noise  = np.expand_dims(res_inv_vars[train_size:train_size+valid_size],-1)*std**2
test_noise   = np.expand_dims(res_inv_vars[train_size+valid_size::],-1)*std**2

train_params = np.asarray(redshifts, dtype=np.float32)[:train_size]
valid_params = np.asarray(redshifts, dtype=np.float32)[train_size:train_size+valid_size]
test_params  = np.asarray(redshifts, dtype=np.float32)[train_size+valid_size::]

train_ras = np.asarray(ras, dtype=np.float32)[:train_size]
valid_ras = np.asarray(ras, dtype=np.float32)[train_size:train_size+valid_size]
test_ras  = np.asarray(ras, dtype=np.float32)[train_size+valid_size::]

train_decs = np.asarray(decs, dtype=np.float32)[:train_size]
valid_decs = np.asarray(decs, dtype=np.float32)[train_size:train_size+valid_size]
test_decs  = np.asarray(decs, dtype=np.float32)[train_size+valid_size::]

train_cat = category[:train_size].astype('float').astype('int')
valid_cat = category[train_size:train_size+valid_size].astype('float').astype('int')
test_cat  = category[train_size+valid_size::].astype('float').astype('int')

train_labels = sublabel[:train_size]
valid_labels = sublabel[train_size:train_size+valid_size]
test_labels  = sublabel[train_size+valid_size::]

train_SN = SNs[:train_size]
valid_SN = SNs[train_size:train_size+valid_size]
test_SN  = SNs[train_size+valid_size::]

train_MJD = MJD[:train_size].astype('float').astype('int')
valid_MJD = MJD[train_size:train_size+valid_size].astype('float').astype('int')
test_MJD = MJD[train_size+valid_size::].astype('float').astype('int')

train_plate_id = plate_id[:train_size].astype('float').astype('int')
valid_plate_id = plate_id[train_size:train_size+valid_size].astype('float').astype('int')
test_plate_id = plate_id[train_size+valid_size::].astype('float').astype('int')

train_fiber = fiber[:train_size].astype('float').astype('int')
valid_fiber = fiber[train_size:train_size+valid_size].astype('float').astype('int')
test_fiber = fiber[train_size+valid_size::].astype('float').astype('int')

In [63]:
print(len(valid_fiber), len(test_fiber))

69820 69822


In [64]:
Supernovas = pd.read_pickle('../forPub/Supernova_Sample.pkl')
df1 = pd.DataFrame()
df1['MJD']   = train_MJD
df1['plate'] = train_plate_id
df1['fiber'] = train_fiber
df1['redshift'] = train_labels
sns= df1.merge(Supernovas,how='inner',on=['plate','MJD','fiber'])
print(len(sns))
df1 = pd.DataFrame()
df1['MJD']   = valid_MJD
df1['plate'] = valid_plate_id
df1['fiber'] = valid_fiber
df1['redshift'] = valid_labels
sns= df1.merge(Supernovas,how='inner',on=['plate','MJD','fiber'])
print(len(sns))
df1 = pd.DataFrame()
df1['MJD']   = test_MJD
df1['plate'] = test_plate_id
df1['fiber'] = test_fiber
df1['redshift'] = test_labels
sns= df1.merge(Supernovas,how='inner',on=['plate','MJD','fiber'])
print(len(sns))

7
3
3


In [65]:
np.savez(os.path.join(root_model_data,'train_%s_new.npy'%label_2),train_data,train_mask,train_noise,train_params,train_ras, train_decs, train_cat, train_labels,train_SN, train_MJD, train_plate_id,train_fiber, mean, std)

In [66]:
np.savez(os.path.join(root_model_data,'valid_%s_new.npy'%label_2),valid_data,valid_mask,valid_noise,valid_params,valid_ras, valid_decs, valid_cat, valid_labels, valid_SN, valid_MJD, valid_plate_id, valid_fiber, mean, std)

In [67]:
np.savez(os.path.join(root_model_data,'test_%s_new.npy'%label_2),test_data,test_mask,test_noise,test_params,test_ras, test_decs, test_cat, test_labels, test_SN, test_MJD, test_plate_id, test_fiber, mean, std)

In [68]:
len(train_data), len(valid_data), len(test_data)

(209462, 69820, 69822)